In [2]:
from sqlalchemy import create_engine, text
import pandas as pd
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
DB_URL = os.environ.get("DB_URL", "sqlite:///mavericks.db")
engine = create_engine(DB_URL)


In [5]:
# via SQL (simple)
df = pd.read_sql(text("SELECT * FROM cars"), engine)

In [6]:
df.head()

,id,listing,link,mileage,price,is_hybrid,date_found
0,1,2024 FORD MAVERICK XLT,https://www.clasificadosonline.com/UDTransDeta...,7332,"$32,995",0,2025-07-22
1,2,FORD MAVERICK 2022,https://www.clasificadosonline.com/UDTransDeta...,0,,0,2025-07-22
2,3,FORD MAVERICK LARIAT FX4 AWD 2022 EQUIPADA!,https://www.clasificadosonline.com/UDTransDeta...,19100,,0,2025-07-22
3,4,FORD MAVERICK HYBRID 2024,https://www.clasificadosonline.com/UDTransDeta...,3,"$29,990",1,2025-07-22
4,5,FORD MAVERICK XL DESDE 27095,https://www.clasificadosonline.com/UDTransDeta...,11,"$27,095",0,2025-07-22


In [7]:
import re
import pandas as pd
from datetime import date

df = df.copy()

def to_int_price(p):
    if not isinstance(p, str): return pd.NA
    digits = re.sub(r"[^\d]", "", p)
    return int(digits) if digits else pd.NA

def to_int_miles(m):
    if not isinstance(m, str): return pd.NA
    digits = re.sub(r"[^\d]", "", m)
    return int(digits) if digits else pd.NA

def extract_year(text):
    if not isinstance(text, str): return pd.NA
    m = re.search(r"\b(20[12]\d)\b", text)  # 2010–2029 quick grab
    return int(m.group(1)) if m else pd.NA

df["price_num"]   = df["price"].map(to_int_price)
df["mileage_num"] = df["mileage"].map(to_int_miles)
df["year"]        = df["listing"].map(extract_year)
df["is_hybrid"]   = df["is_hybrid"].astype("boolean")
df["days_on_market"] = (pd.Timestamp.today().normalize() - pd.to_datetime(df["date_found"])).dt.days
df.head()


,id,listing,link,mileage,price,is_hybrid,date_found,price_num,mileage_num,year,days_on_market
0,1,2024 FORD MAVERICK XLT,https://www.clasificadosonline.com/UDTransDeta...,7332,"$32,995",False,2025-07-22,32995,7332,2024,20
1,2,FORD MAVERICK 2022,https://www.clasificadosonline.com/UDTransDeta...,0,,False,2025-07-22,<NA>,0,2022,20
2,3,FORD MAVERICK LARIAT FX4 AWD 2022 EQUIPADA!,https://www.clasificadosonline.com/UDTransDeta...,19100,,False,2025-07-22,<NA>,19100,2022,20
3,4,FORD MAVERICK HYBRID 2024,https://www.clasificadosonline.com/UDTransDeta...,3,"$29,990",True,2025-07-22,29990,3,2024,20
4,5,FORD MAVERICK XL DESDE 27095,https://www.clasificadosonline.com/UDTransDeta...,11,"$27,095",False,2025-07-22,27095,11,<NA>,20


In [8]:
hybrids = df.query("is_hybrid == True").copy()
hybrids_sorted = hybrids.sort_values(["price_num", "mileage_num"], ascending=[True, True])

cols = ["listing", "year", "price", "mileage_num", "days_on_market", "link"]
hybrids_sorted[cols].head(50)  # show top 50; adjust as you like


,listing,year,price,mileage_num,days_on_market,link
20,2023 Ford Maverick XLT Hibrida,2023,"$27,100",31000,20,https://www.clasificadosonline.com/UDTransDeta...
119,"Ford Maverick XLT hybrid 2023 | 37,036 | CPO",2023,"$27,990",37036,20,https://www.clasificadosonline.com/UDTransDeta...
27,NUEVA Ford Maverick Hybrid 2024 Cactus Gray,2024,"$28,995",0,20,https://www.clasificadosonline.com/UDTransDeta...
29,NUEVA Ford Maverick XL Hybrid 2024 LIQUIDACIÓ,2024,"$28,995",0,20,https://www.clasificadosonline.com/UDTransDeta...
19,Ford Maverick XL Hybrid 2024,2024,"$29,871",<NA>,20,https://www.clasificadosonline.com/UDTransDeta...
160,MAVERIK XL HYBRID 2024,2024,"$29,891",14,20,https://www.clasificadosonline.com/UDTransDeta...
30,Ford Maverick 2024 XL Hybrida ShadowBlack,2024,"$29,987",<NA>,20,https://www.clasificadosonline.com/UDTransDeta...
3,FORD MAVERICK HYBRID 2024,2024,"$29,990",3,20,https://www.clasificadosonline.com/UDTransDeta...
84,FORD MAVERICK XL HYBRID 2024,2024,"$29,990",3,20,https://www.clasificadosonline.com/UDTransDeta...
238,Ford Maverick 2024 XL Hybrida ShadowBlack,2024,"$30,989",<NA>,1,https://www.clasificadosonline.com/UDTransDeta...


In [9]:
PRICE_CAP = 30000
AGED_DAYS = 14

aged_hybrids = hybrids_sorted.query("days_on_market >= @AGED_DAYS")
aged_under_cap = aged_hybrids.query("price_num.notna() and price_num <= @PRICE_CAP")

print("Hybrids total:", len(hybrids))
print("Aged hybrids (>=14d):", len(aged_hybrids))
print("Aged hybrids under $30k:", len(aged_under_cap))

aged_under_cap[cols].head(25)


Hybrids total: 26
Aged hybrids (>=14d): 22
Aged hybrids under $30k: 9


,listing,year,price,mileage_num,days_on_market,link
20,2023 Ford Maverick XLT Hibrida,2023,"$27,100",31000,20,https://www.clasificadosonline.com/UDTransDeta...
119,"Ford Maverick XLT hybrid 2023 | 37,036 | CPO",2023,"$27,990",37036,20,https://www.clasificadosonline.com/UDTransDeta...
27,NUEVA Ford Maverick Hybrid 2024 Cactus Gray,2024,"$28,995",0,20,https://www.clasificadosonline.com/UDTransDeta...
29,NUEVA Ford Maverick XL Hybrid 2024 LIQUIDACIÓ,2024,"$28,995",0,20,https://www.clasificadosonline.com/UDTransDeta...
19,Ford Maverick XL Hybrid 2024,2024,"$29,871",<NA>,20,https://www.clasificadosonline.com/UDTransDeta...
160,MAVERIK XL HYBRID 2024,2024,"$29,891",14,20,https://www.clasificadosonline.com/UDTransDeta...
30,Ford Maverick 2024 XL Hybrida ShadowBlack,2024,"$29,987",<NA>,20,https://www.clasificadosonline.com/UDTransDeta...
3,FORD MAVERICK HYBRID 2024,2024,"$29,990",3,20,https://www.clasificadosonline.com/UDTransDeta...
84,FORD MAVERICK XL HYBRID 2024,2024,"$29,990",3,20,https://www.clasificadosonline.com/UDTransDeta...


In [10]:
from fuzzywuzzy import fuzz

def looks_hybrid(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    return (
        fuzz.partial_ratio(t, "hybrid")  >= 70 or
        fuzz.partial_ratio(t, "hibrido") >= 70 or
        "híbrido" in t or               # accent exact match
        "hibrida" in t or               # feminine spelling
        "híbrida" in t or
        "hev" in t or                   # common shorthand
        "hyb" in t                      # very loose
    )

df["hybrid_from_text"] = df["listing"].map(looks_hybrid)
print("DB is_hybrid True count:", int(df["is_hybrid"].sum(skipna=True)))
print("Text-derived hybrid count:", int(df["hybrid_from_text"].sum()))
df.loc[df["hybrid_from_text"] & ~df["is_hybrid"], ["listing","price","link"]].head(10)


DB is_hybrid True count: 26
Text-derived hybrid count: 26


c:\Users\Erik\PycharmProjects\day-93-custom-web-scraper\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,listing,price,link


In [11]:
cols = ["listing","price","mileage","date_found","link"]

print("== current hybrids ==")
display(df.loc[df["is_hybrid"] == True, cols].head(20))

print("== rows that mention hyb-like terms in listing (not flagged) ==")
mask_near = df["listing"].str.contains(r"hyb|h[ií]br[ií]d", case=False, na=False)
display(df.loc[mask_near & (df["is_hybrid"] != True), cols].head(20))


== current hybrids ==


,listing,price,mileage,date_found,link
3,FORD MAVERICK HYBRID 2024,"$29,990",3,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
18,Ford Maverick XL Hibrida 2.5 Premium nueva,,10,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
19,Ford Maverick XL Hybrid 2024,"$29,871",,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
20,2023 Ford Maverick XLT Hibrida,"$27,100",31000,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
25,Ford Maverick XL Hybrid 24,"$35,995",16,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
27,NUEVA Ford Maverick Hybrid 2024 Cactus Gray,"$28,995",0,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
29,NUEVA Ford Maverick XL Hybrid 2024 LIQUIDACIÓ,"$28,995",0,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
30,Ford Maverick 2024 XL Hybrida ShadowBlack,"$29,987",,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
56,Ford Maverick XLT Hybrid 2024,"$31,990",,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...
62,NUEVA Ford Maverick XLT Hybrid 2024 Sunroof,"$34,995",0,2025-07-22,https://www.clasificadosonline.com/UDTransDeta...


== rows that mention hyb-like terms in listing (not flagged) ==


,listing,price,mileage,date_found,link


In [12]:
# Save full dataset to CSV
df.to_csv("maverick_listings.csv", index=False, encoding="utf-8-sig")
print("✅ CSV saved as maverick_listings.csv")


✅ CSV saved as maverick_listings.csv


In [ ]:
# all hybrids, cheapest first
hybrids_sorted = (
    df[df["is_hybrid"] == True]
    .sort_values(["price_num","mileage_num"], ascending=[True, True])
)
hybrid_rows_html = "".join(
    f"<tr><td>{r['listing']}</td><td>{r['price']}</td><td>{r['mileage']}</td>"
    f"<td>{r['days_on_market']}</td><td><a href='{r['link']}'>link</a></td></tr>"
    for _, r in hybrids_sorted.head(12).iterrows()
)

# aged hybrids under cap
PRICE_CAP, AGED_DAYS = 30000, 14
aged_under_cap = hybrids_sorted.query("price_num.notna() and price_num<=@PRICE_CAP and days_on_market>=@AGED_DAYS")
aged_rows_html = "".join(
    f"<tr><td>{r['listing']}</td><td>{r['price']}</td><td>{r['mileage']}</td>"
    f"<td>{r['days_on_market']}</td><td><a href='{r['link']}'>link</a></td></tr>"
    for _, r in aged_under_cap.head(12).iterrows()
)
